In [17]:
from onnx import TensorProto
from qonnx.core.modelwrapper import ModelWrapper
from slice_template import slice_node

tmp_model = ModelWrapper("tinyyolo_infershapes.onnx")

update_dict = {}
key_list = ["input","starts","ends","axes","splits","output"]

for s in tmp_model.get_nodes_by_op_type("Slice"):
    node_inx = tmp_model.get_node_index(s)

    tmp_model.get_node_index(slice_nodes[0])

    vinfo = [tmp_model.get_tensor_valueinfo(inp) for inp in s.input]
    init = [tmp_model.get_initializer(inp) for inp in s.input]
    shapes = [tmp_model.get_tensor_shape(i) for i in s.input]
    t_names = [i for i in s.input]
    
    attr = {}
    for k in key_list:
        attr[k] = [None,None,None]

    for ind in range(len(shapes)):
        attr[key_list[ind]] = [t_names[ind],shapes[ind],init[ind]]
    for i in s.output: 
        attr["output"] = [i, tmp_model.get_tensor_shape(i)]
    print("*"*110)
    print(attr)
    x = slice_node(input_shape=attr["input"][1],output_shape=attr["output"][1],
                   starts_value=attr["starts"][2], ends_value=attr["ends"][2], axes_value=attr["axes"][2],
                   input_tensor=attr["input"][0], output_tensor=attr["output"][0],
                  dtype=TensorProto.INT64)
    update_dict[s.name] = [node_inx,x.make_node()]
    x.make_model(s.name+".onnx")

**************************************************************************************************************
{'input': ['735', [4], None], 'starts': ['737', [1], array([0])], 'ends': ['738', [1], array([2])], 'axes': ['736', [1], array([0])], 'splits': [None, None, None], 'output': ['739', [2]]}
**************************************************************************************************************
{'input': ['673', [], None], 'starts': ['675', [1], array([0])], 'ends': ['676', [1], array([2])], 'axes': ['674', [1], array([4])], 'splits': ['677', [1], array([1])], 'output': ['678', []]}
**************************************************************************************************************
{'input': ['673', [], None], 'starts': ['686', [1], array([2])], 'ends': ['687', [1], array([4])], 'axes': ['685', [1], array([4])], 'splits': ['688', [1], array([1])], 'output': ['689', []]}
***********************************************************************************************

In [27]:
model = ModelWrapper("tinyyolo-20210831.onnx")
nodes = model.graph.node
model.graph
# for k,val in update_dict.items():
type(model.graph.node)

google.protobuf.pyext._message.RepeatedCompositeContainer